In [2]:
#-------------------------------
#Stocktwitts scraper
#-------------------------------






# ---- Connect  mysqlconnector.py ----
with open("mysqlconnector.py") as f:
    code = compile(f.read(), "mysqlconnector.py", "exec")
    exec(code, globals())
    print("⚡ Executed: mysqlconnector.py")


# ---- Connect  selenium_setup.py ----
with open("selenium_setup.py") as f:
    code = compile(f.read(), "selenium_setup.py", "exec")
    exec(code, globals())
    print("⚡ Executed: selenium_setup.py")






run_sql('use algo_trader')


driver = create_browser(headless = False)


#Login section
#----------------------------------------------------------------------------------------------------------------------------------------------
driver.get('https://stocktwits.com/signin')

time.sleep(3)


email_input = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div[1]/div[3]/div/div[2]/form/div[1]/input')
email_input.send_keys("deanemarks1@gmail.com")


password_input = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div[1]/div[3]/div/div[2]/form/div[2]/input')
password_input.send_keys("Redwolves16" + Keys.ENTER)




submit_button = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div[1]/div[3]/div/div[2]/form/button')
submit_button.click()



# wait for page to load
time.sleep(2)

# find the Log In button using its data-testid
login_button = driver.find_element(By.CSS_SELECTOR, "button[data-testid='log-in-submit']")

# click it
login_button.click()
#Login section
#----------------------------------------------------------------------------------------------------------------------------------------------



time.sleep()





db_table_list = [
    'stock_twitts__trending', 
    'stock_twitts__most_active',
    'stock_twitts__most_new_watchers', 
    'stock_twitts__top_gainers',
    'stock_twitts__top_loosers'
]



url_list = [
        'https://stocktwits.com/sentiment',
        'https://stocktwits.com/sentiment/most-active',
        'https://stocktwits.com/sentiment/watchers',
        'https://stocktwits.com/sentiment/top-gainers',
        'https://stocktwits.com/sentiment/top-losers'
]
        



Y = 0
while Y < len(db_table_list):
        
    
    driver.get(url_list[Y])
    
    
    time.sleep(.3)  # wait for JS to render rows
    
    
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    
    # Extract all relevant div elements
    all_data = soup.find_all('div', class_='TickerTable_bodyCell__SZDgb')
    
    # Extract and clean text from each div
    all_data_text = [div.get_text(strip=True) for div in all_data if div.get_text(strip=True)]
    
    
    
    
     
    ticker_elements = soup.find_all('p', class_='TickerTable_symbol___hQ3F font-semibold')
    ticker_column = [el.get_text(strip=True) for el in ticker_elements]
    
    
    volume_column = [parse_numeric(v) for v in all_data_text[4::8]]
    
    
    fifty_two_week_high = all_data_text[5::8]
    fifty_two_week_high_column = [clean_price(val) for val in fifty_two_week_high]
    
    
    fifty_two_week_low = all_data_text[6::8]
    fifty_two_week_low_column = [clean_price(val) for val in fifty_two_week_low]
    
    
    market_cap = all_data_text[7::8]
    market_cap_column = [int(parse_numeric(val)) for val in market_cap]
    
    
    
    price_elements = soup.find_all('div', class_='TickerTable_watch__Q72pY')
    price_column = [clean_price(div.get_text(strip=True)) for div in price_elements if div.get_text(strip=True)]
    
    
    
    
    # Parse Percent page
    #######################################################################################################################
    # Get percent change elements
    percent_change_elements = soup.find_all(
        'div',
        class_='cursor-pointer TickerTable_bodyCell__SZDgb relative py-3 pl-1 justify-center grow-1 text-right text-sm border-t border-primary-hairline h-full'
    )
    
    # Parse numbers and sign from class names
    percent_change_column = []
    
    for el in percent_change_elements[1::2]:
        text = el.get_text(strip=True).replace('%', '')
        try:
            value = float(text)
        except ValueError:
            value = 0.0
    
        # Look for nested span with class that indicates positive/negative
        span = el.find('span')
        if span:
            class_list = ' '.join(span.get('class', []))
            if 'Change_negative' in class_list:
                value *= -1
    
        percent_change_column.append(value)
    #######################################################################################################################
    
    
    
    
    
    
    
    
    
    #Put it all together
    #####################################################################
    row_count = min(
        len(ticker_column),
        len(price_column),
        len(percent_change_column),
        len(volume_column),
        len(fifty_two_week_high_column),
        len(fifty_two_week_low_column),
        len(market_cap_column)
    )
    
    from datetime import datetime
    
    # Add current timestamp
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    # Trim all columns to the shortest length
    df = pd.DataFrame({
        'ticker': ticker_column[:row_count],
        'price': price_column[:row_count],
        'change_percent': percent_change_column[:row_count],
        'volume': volume_column[:row_count],
        'fifty_two_week_high': fifty_two_week_high_column[:row_count],
        'fifty_two_week_low': fifty_two_week_low_column[:row_count],
        'market_cap': market_cap_column[:row_count],
        'timestamp': [timestamp] * row_count
    })
    
    ####################################################################
    
    
    
    
    run_sql(f"""
        INSERT INTO algo_trader.{db_table_list[Y]} 
        (ticker, price, change_percent, volume, fifty_two_week_high, fifty_two_week_low, market_cap, timestamp)
        VALUES (
            '{row['ticker']}',
            {row['price']},
            {row['change_percent']},
            {row['volume']},
            {row['fifty_two_week_high']},
            {row['fifty_two_week_low']},
            {row['market_cap']},
            '{row['timestamp']}'
        )
    """)

    


    

    print(Y)


    Y+=1

print(f"✅ Scrape complete \n")



🤖 Bot Initiated



KeyboardInterrupt: 